Some setup notes and Help - 
Patrick Connelly
pconnell89@gmail.com

Getting Fair Face Up-and-running

To get all of the data to start some work, I began by downloading some files and storing them in a folder structure.

Spreadsheet/CSV is located here: https://github.com/Raschka-research-group/coral-cnn/tree/master/datasets.  I believe I was using the afad_test csv.

The images themselves for AFAD (Asian Face Age Dataset) are tarballs located in a separate repo on github: https://github.com/John-niu-07/tarball.  To decompress / extract them, you can use 7-zip on windows, or tar xvf filename_here on linux or mac.

Some other dependencies - 

    1) Install windows subsystem for linux (WSL)

    2) Simple method for Windows - install miniconda on WSL

        - enter WSL shell
    
        - wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    
        - bash Miniconda3-latest-Linux-x86_64.sh
    
        - rm Miniconda3-latest-Linux-x86_64.sh
    
        - conda init
    
        - conda activate

    3) Create a python env within conda (https://learn.microsoft.com/en-us/windows/ai/directml/gpu-pytorch-windows)

        - conda create --name directml python=3.8 ipython (or whatever you want) 

        - conda activate directml (to enter the environment)

        - Within the environment - 

            a) Install Python 3.8

            b) Pip install torch - https://pytorch.org/get-started/locally/

            c) Pip torch_directml, numpy, pandas, dlib, deepface

Overall purpose of the code in this file is to identify locally stored and unzipped data, and use that to filter down the spreadsheet of data from a source dataset. 
The output provides you with a csv called "img_paths" which can be fed to deepFaceIteration.py or to predict.py as a parameter for iteration.



In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
afad_test = pd.read_csv('C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\afad_test.csv')


Below, I pull in the CSV from the Asian Face Dataset.  I downloaded the CSV and one chunk of the images locally.

This first block below is just to test and verify that I was able to bring in the information.

In [3]:
afad_test = pd.read_csv('C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\afad_test.csv')
afad_test.head(7)

,file,path,age,gender
0,406208-0.jpg,39/112/406208-0.jpg,24,female
1,470804-0.jpg,39/112/470804-0.jpg,24,female
2,411657-2.jpg,39/112/411657-2.jpg,24,female
3,471528-2.jpg,39/112/471528-2.jpg,24,female
4,398608-1.jpg,39/112/398608-1.jpg,24,female
5,470572-0.jpg,39/112/470572-0.jpg,24,female
6,401238-0.jpg,39/112/401238-0.jpg,24,female


Spreadsheet/CSVs were located here on the web: https://github.com/Raschka-research-group/coral-cnn/tree/master/datasets 

The images themselves are tarballs located in a separate repo on github: https://github.com/John-niu-07/tarball.

In terms of this dataset - the images are "chunked" into tarball/zip files in GitHub for the AFAD dataset.  The spreadsheet/csv, however, has a listing of ALL file paths for all files in the dataset.  In order to get to a point where I could only iterate over the target files, I had to do a directory walk to identify only .jpg files, searching recursively through a local folder.  

This folder ahd the repo / files from a git clone of FairFace, models downloaded separately from Google Drive (not in the GitHub repo directly), the chunk of images from the AFAD dataface extracted into two subfolders, and the full spreadsheet of all items in the AFAD dataset.

So I did the directory walk to get a list of all JPGs.  I left-joined the source spreadsheet (left table) to the present files (right table), and then filtered where the right table was blank/NA, so that the result was a table only of the available files for iteration, and then wrote that out to a csv.

I used this to build my input for my deepFaceIteration.py script and the modified version of predict.py from the FairFace model.

I didn't set this up with functions or parameters (i.e. tell me where you wanna save the file).  

In [4]:
PATH = "C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full"
EXT = "*.jpg"

# get a list of all filenames recursively from the current directory in which this script runs
# and down into any subfolders as well

all_jpg = [file 
           for path, subdir, files in os.walk(PATH)
           for file in glob(os.path.join(path,EXT))]

#can be used to verify how many you identified in the local folder(s)
#len(all_jpg)
#all_jpg[0]

present_files = pd.DataFrame({'jpgs': all_jpg})
present_files['file']=(present_files['jpgs'].str.split('\\')).str[-1]
present_files

#do a left join between the files we know could exist (left) and the files we found (right)
#filter the list down only to items where there was a match as well (maybe inner join is smarter)
merged=afad_test.merge(present_files,on='file',how='left')
merged = merged[merged['jpgs'].notna()]

#trim and generate the output file!
merged['jpgs'] = merged['jpgs'].str.replace('C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full','.')
merged['img_path'] = merged['jpgs']
merged=merged.drop(labels=['jpgs','path','file','age','gender'],axis=1)

merged.to_csv('C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\afad_loc_clean.csv',index=False)

len(merged)

491